In [10]:
from lsst.ts.observing import ObservingBlock, ObservingScript

from sqlalchemy import create_engine
from datetime import datetime 
from pytz import timezone
import pandas as pd

import matplotlib
import matplotlib.pyplot as pyplot

In [11]:
engine = create_engine('sqlite:////sdf/data/rubin/shared/ops-rehearsal-3/scheduler_sims/ops_rehearsal_apr_2024_v3_playbackBlock.db')
Observations = pd.read_sql_table('observations', engine)

In [48]:
from astropy.time import Time
# Summarize and then select columns we want to use

print(Observations.columns)

# Make a df with selected values.

visit_info = Observations[['observationId', 'observationStartMJD', 
                           'note', 'playbackBlock']].query('playbackBlock >= 208 and playbackBlock < 258').copy()

chile_time = timezone('America/Santiago')
local_time = Time(visit_info['observationStartMJD'], format='mjd').to_datetime(timezone=chile_time)
visit_info['observationStartLocalTime'] = local_time

days = [time.day for time in local_time]
visit_info['day'] = days

# Print updated dataframe
visit_info


Index(['observationId', 'fieldRA', 'fieldDec', 'observationStartMJD',
       'flush_by_mjd', 'visitExposureTime', 'filter', 'rotSkyPos',
       'rotSkyPos_desired', 'numExposures', 'airmass', 'seeingFwhm500',
       'seeingFwhmEff', 'seeingFwhmGeom', 'skyBrightness', 'night', 'slewTime',
       'visitTime', 'slewDistance', 'fiveSigmaDepth', 'altitude', 'azimuth',
       'paraAngle', 'cloud', 'moonAlt', 'sunAlt', 'note', 'target', 'fieldId',
       'proposalId', 'block_id', 'observationStartLST', 'rotTelPos',
       'rotTelPos_backup', 'moonAz', 'sunAz', 'sunRA', 'sunDec', 'moonRA',
       'moonDec', 'moonDistance', 'solarElong', 'moonPhase', 'cummTelAz',
       'scripted_id', 'playbackBlock'],
      dtype='object')


,observationId,observationStartMJD,note,playbackBlock,observationStartLocalTime,day
12265,12265,60402.979361,Rubin_SV_125_-15,208,2024-04-02 20:30:16.825781-03:00,2
12266,12266,60402.979762,Rubin_SV_125_-15,208,2024-04-02 20:30:51.468188-03:00,2
12267,12267,60402.980160,Rubin_SV_125_-15,208,2024-04-02 20:31:25.801533-03:00,2
12268,12268,60402.980561,Rubin_SV_125_-15,208,2024-04-02 20:32:00.479516-03:00,2
12269,12269,60402.980964,Rubin_SV_125_-15,208,2024-04-02 20:32:35.282535-03:00,2
...,...,...,...,...,...,...
15154,15154,60405.417853,Rubin_SV_280_-48,257,2024-04-05 07:01:42.473404-03:00,5
15155,15155,60405.418250,Rubin_SV_280_-48,257,2024-04-05 07:02:16.793485-03:00,5
15156,15156,60405.418652,Rubin_SV_280_-48,257,2024-04-05 07:02:51.495031-03:00,5
15157,15157,60405.419050,Rubin_SV_280_-48,257,2024-04-05 07:03:25.880454-03:00,5


In [58]:
#Group the data frame by block and print how many are in each block.
days = visit_info.groupby(['day'])

tuesday = days.get_group((2,))
wednesday = days.get_group((3,))
thursday= days.get_group((4,))
friday= days.get_group((5,))

tuesday

,observationId,observationStartMJD,note,playbackBlock,observationStartLocalTime,day
12265,12265,60402.979361,Rubin_SV_125_-15,208,2024-04-02 20:30:16.825781-03:00,2
12266,12266,60402.979762,Rubin_SV_125_-15,208,2024-04-02 20:30:51.468188-03:00,2
12267,12267,60402.980160,Rubin_SV_125_-15,208,2024-04-02 20:31:25.801533-03:00,2
12268,12268,60402.980561,Rubin_SV_125_-15,208,2024-04-02 20:32:00.479516-03:00,2
12269,12269,60402.980964,Rubin_SV_125_-15,208,2024-04-02 20:32:35.282535-03:00,2
...,...,...,...,...,...,...
12580,12580,60403.122125,Rubin_SV_225_-40,213,2024-04-02 23:55:51.583313-03:00,2
12581,12581,60403.122522,Rubin_SV_225_-40,213,2024-04-02 23:56:25.908870-03:00,2
12582,12582,60403.122919,Rubin_SV_225_-40,213,2024-04-02 23:57:00.170652-03:00,2
12583,12583,60403.123316,Rubin_SV_225_-40,213,2024-04-02 23:57:34.542629-03:00,2


In [50]:
tuesday.groupby(['playbackBlock']).first()

,observationId,observationStartMJD,note,observationStartLocalTime,day
playbackBlock,,,,,
208,12265,60402.979361,Rubin_SV_125_-15,2024-04-02 20:30:16.825781-03:00,2
209,12325,60403.006666,COSMOS,2024-04-02 21:09:35.919606-03:00,2
210,12385,60403.034071,Rubin_SV_125_-15,2024-04-02 21:49:03.745522-03:00,2
211,12445,60403.061554,COSMOS,2024-04-02 22:28:38.225212-03:00,2
212,12505,60403.088496,DESI_SV3_R1,2024-04-02 23:07:26.060181-03:00,2
213,12565,60403.116148,Rubin_SV_225_-40,2024-04-02 23:47:15.183712-03:00,2


In [ ]:
wednesday.groupby(['playbackBlock']).first()

In [ ]:
thursday.groupby(['playbackBlock']).first()

In [ ]:
friday.groupby(['playbackBlock']).first()

In [60]:
visit_info.groupby(['playbackBlock']).first()

,observationId,observationStartMJD,note,observationStartLocalTime,day
playbackBlock,,,,,
208,12265,60402.979361,Rubin_SV_125_-15,2024-04-02 20:30:16.825781-03:00,2
209,12325,60403.006666,COSMOS,2024-04-02 21:09:35.919606-03:00,2
210,12385,60403.034071,Rubin_SV_125_-15,2024-04-02 21:49:03.745522-03:00,2
211,12445,60403.061554,COSMOS,2024-04-02 22:28:38.225212-03:00,2
212,12505,60403.088496,DESI_SV3_R1,2024-04-02 23:07:26.060181-03:00,2
213,12565,60403.116148,Rubin_SV_225_-40,2024-04-02 23:47:15.183712-03:00,2
214,12625,60403.143907,DESI_SV3_R1,2024-04-03 00:27:13.556450-03:00,3
215,12685,60403.171399,Rubin_SV_225_-40,2024-04-03 01:06:48.881835-03:00,3
216,12745,60403.198432,DEEP_A0,2024-04-03 01:45:44.487429-03:00,3
